<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模型
假設每個輸入項xi不再只是單一數值，而是一個由xil,...,xik所組成的向量k。  
多元遞歸模型架設如下：


```
yi = α+β₁χɪʟ+...+βkχɪʟ
```

